# DSC 540 Project Milestone 4

Logan Quandt

In [464]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import requests
import json
import os
import urllib.request
import glom

In [873]:
#define URL and open using urllib.request
url = "https://restcountries.com/v2/all"
response = urllib.request.urlopen(url)

In [874]:
#read and decode response
data = response.read().decode()

In [875]:
#convert JSON to dict using JSON load
data_json = json.loads(data)

# 1: Format data into a more readable format

In [876]:
#convert JSON data to Pandas Dataframe for readability
df = pd.json_normalize(data_json)
df.head()

,name,topLevelDomain,alpha2Code,alpha3Code,callingCodes,capital,altSpellings,subregion,region,population,...,translations.nl,translations.hr,translations.fa,translations.de,translations.es,translations.fr,translations.ja,translations.it,translations.hu,gini
0,Afghanistan,[.af],AF,AFG,[93],Kabul,"[AF, Afġānistān]",Southern Asia,Asia,40218234,...,Afghanistan,Afganistan,افغانستان,Afghanistan,Afganistán,Afghanistan,アフガニスタン,Afghanistan,Afganisztán,NaN
1,Åland Islands,[.ax],AX,ALA,[358],Mariehamn,"[AX, Aaland, Aland, Ahvenanmaa]",Northern Europe,Europe,28875,...,Ålandeilanden,Ålandski otoci,جزایر الند,Åland,Alandia,Åland,オーランド諸島,Isole Aland,Åland-szigetek,NaN
2,Albania,[.al],AL,ALB,[355],Tirana,"[AL, Shqipëri, Shqipëria, Shqipnia]",Southern Europe,Europe,2837743,...,Albanië,Albanija,آلبانی,Albanien,Albania,Albanie,アルバニア,Albania,Albánia,33.2
3,Algeria,[.dz],DZ,DZA,[213],Algiers,"[DZ, Dzayer, Algérie]",Northern Africa,Africa,44700000,...,Algerije,Alžir,الجزایر,Algerien,Argelia,Algérie,アルジェリア,Algeria,Algéria,27.6
4,American Samoa,[.as],AS,ASM,[1],Pago Pago,"[AS, Amerika Sāmoa, Amelika Sāmoa, Sāmoa Amelika]",Polynesia,Oceania,55197,...,Amerikaans Samoa,Američka Samoa,ساموآی آمریکا,Amerikanisch-Samoa,Samoa Americana,Samoa américaines,アメリカ領サモア,Samoa Americane,Amerikai Szamoa,NaN


In [877]:
#drop unneeded columns to make dataframe cleaner and easier to work with
df = df.drop(['topLevelDomain', 'alpha2Code', "alpha3Code", "callingCodes", "altSpellings", 'population', 'demonym'], axis=1)
df = df.drop(['numericCode', 'translations.pt', 'translations.nl', 'translations.hr'], axis=1)
df = df.drop(['translations.fa', 'translations.de', 'translations.es', 'translations.fr', 'translations.ja', 'translations.it', 'translations.hu'], axis=1)
df = df.drop (['timezones', 'borders', 'flags.png', 'flags.svg', 'nativeName', 'flag','regionalBlocs', 'translations.br','currencies',"languages"], axis=1)

In [878]:
#view dataframe to ensure unwanted columns were dropped
df

,name,capital,subregion,region,latlng,area,cioc,independent,gini
0,Afghanistan,Kabul,Southern Asia,Asia,"[33.0, 65.0]",652230.0,AFG,True,NaN
1,Åland Islands,Mariehamn,Northern Europe,Europe,"[60.116667, 19.9]",1580.0,NaN,False,NaN
2,Albania,Tirana,Southern Europe,Europe,"[41.0, 20.0]",28748.0,ALB,True,33.2
3,Algeria,Algiers,Northern Africa,Africa,"[28.0, 3.0]",2381741.0,ALG,True,27.6
4,American Samoa,Pago Pago,Polynesia,Oceania,"[-14.33333333, -170.0]",199.0,ASA,False,NaN
...,...,...,...,...,...,...,...,...,...
245,Wallis and Futuna,Mata-Utu,Polynesia,Oceania,"[-13.3, -176.2]",142.0,NaN,False,NaN
246,Western Sahara,El Aaiún,Northern Africa,Africa,"[24.5, -13.0]",266000.0,NaN,False,NaN
247,Yemen,Sana'a,Western Asia,Asia,"[15.0, 48.0]",527968.0,YEM,True,36.7
248,Zambia,Lusaka,Eastern Africa,Africa,"[-15.0, 30.0]",752618.0,ZAM,True,57.1


# 2: Identify outliers and bad data

In [879]:
#check shape of dataframe before identifying bad data
df.shape

(250, 9)

In [880]:
#get count of NAs by column
df.isna().sum()

name            0
capital         5
subregion       0
region          0
latlng          1
area           10
cioc           45
independent     0
gini           83
dtype: int64

In [881]:
#drop missing data
df = df.dropna()

In [882]:
#check sum of NAs reamining after dropping
df.isna().sum()

name           0
capital        0
subregion      0
region         0
latlng         0
area           0
cioc           0
independent    0
gini           0
dtype: int64

# 3: Format data into a more readable format - Fix column with list formatting

In [883]:
#check sum of NAs remaining
print(df.isna().sum())
print(df.shape)

name           0
capital        0
subregion      0
region         0
latlng         0
area           0
cioc           0
independent    0
gini           0
dtype: int64
(165, 9)


In [884]:
#split column in list format to two columns
split_df = pd.DataFrame(df['latlng'].tolist(), columns=['Latitude', 'Longitude'])

In [885]:
#concate dataframes together
df = pd.concat([df, split_df], axis=1)

In [886]:
#drop latlng column and keep new latitude and longitude columns
df = df.drop(['latlng'],axis=1)
df.head()

,name,capital,subregion,region,area,cioc,independent,gini,Latitude,Longitude
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41.0,20.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.0,3.0
2,Albania,Tirana,Southern Europe,Europe,28748.0,ALB,True,33.2,-12.5,18.5
3,Algeria,Algiers,Northern Africa,Africa,2381741.0,ALG,True,27.6,-34.0,-64.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.0,45.0


In [887]:
#drop nas after concating
df = df.dropna()

In [888]:
#check shape after dropping NAs
df.shape

(107, 10)

# 4: Replace Headers

In [889]:
#check current column names
df.columns

Index(['name', 'capital', 'subregion', 'region', 'area', 'cioc', 'independent',
       'gini', 'Latitude', 'Longitude'],
      dtype='object')

In [890]:
#replace headers using set_axis
df = df.set_axis(['Country Name','Capital City', 'Subregion', 'World Region', 'Area', 'Abbreviation', 
                                       'Independent or Dependency', 'GINI Index', 'Latitude', 'Longitude'], axis=1)

In [891]:
#check to ensure headers were replaced
df.head()

,Country Name,Capital City,Subregion,World Region,Area,Abbreviation,Independent or Dependency,GINI Index,Latitude,Longitude
2,Albania,Tirana,Southern Europe,Europe,28748.0,ALB,True,33.2,-12.500000,18.500000
3,Algeria,Algiers,Northern Africa,Africa,2381741.0,ALG,True,27.6,-34.000000,-64.000000
6,Angola,Luanda,Middle Africa,Africa,1246700.0,ANG,True,51.3,47.333333,13.333333
10,Argentina,Buenos Aires,South America,Americas,2780400.0,ARG,True,42.9,50.833333,4.000000
11,Armenia,Yerevan,Western Asia,Asia,29743.0,ARM,True,29.9,17.250000,-88.750000


# 5: Find duplicates

In [892]:
#check for duplicates using duplicated
df['Country Name'].duplicated().any()

False